In [3]:
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
%matplotlib inline

In [4]:
data = pd.read_csv('new.csv', delimiter=';', encoding='utf-8', on_bad_lines='skip')
data

,id,feedback_text,answer_text
0,5,Töğirlab bera olasizlami,Assalomu alaykum. Savolingizni aniqroq yozib y...
1,6,Pul oʻtkazayotgandim pul oʻtyabdi deb turibdi,"Assalomu alaykum. Hurmatli mijoz, to'lovingiz ..."
2,15,Firibgarlar kim,Assalomu alaykum. Savolingizni aniqroq yozib y...
3,18,Mani shotimdan pul yecildi digan xabar keldi l...,"Assalomu alaykum. Xozir siz bilan bog'lanib, y..."
4,23,Pul tushmadi ku,Assalomu alaykum. Savolingizni aniqroq yozib y...
...,...,...,...
15323,152176,Click uz2010,Assalomu alaykum. Qanday yordam bera olaman?
15324,152188,Ассалому алейкум,Assalomu alaykum. Qanday yordam bera olaman?
15325,152206,M,Assalomu alaykum. Qanday yordam bera olaman?
15326,152209,/start support_chat,Assalomu alaykum. Qanday yordam bera olaman?


In [8]:
top_23_answers = data.groupby('answer_text').count().sort_values('feedback_text', ascending=False).head(23).drop('id', axis=1).reset_index()

In [9]:
top_23_answers = top_23_answers.rename(columns={'feedback_text': 'count'})
top_23_answers

,answer_text,count
0,Assalomu alaykum. Savolingizni aniqroq yozib y...,1529
1,"Assalomu alaykum. Xozir siz bilan bog'lanib, y...",1374
2,"Assalomu alaykum. Hurmatli mijoz, to'lov tafsi...",1299
3,Assalomu alaykum. Bizda “To'lovlar ...,1172
4,Assalomu alaykum. Aynan qanday xatolik bo'lmoqda?,942
5,Assalomu alaykum. Agar kartangiz bizning tizim...,864
6,Assalomu alaykum. Mablag'ni yuborgan mijozimiz...,851
7,Salomat bo'ling.,839
8,Assalaamu alaykum. A garda Click Pin kodingizn...,828
9,Assalomu alaykum. Bizga hamyoni foallashtirish...,816


In [10]:
top_23_answers['count'].sum()

15323

In [11]:
classess = top_23_answers['answer_text'].tolist()

In [12]:
classess

['Assalomu alaykum. Savolingizni aniqroq yozib yuboring iltimos.',
 "Assalomu alaykum. Xozir siz bilan bog'lanib, yordam beramiz.",
 "Assalomu alaykum. Hurmatli mijoz, to'lov tafsilotlarini (To'lov raqami, summasi, vaqti va hokazo) bizga yuboring, tekshirib ko'ramiz.",
 "           Assalomu alaykum. Bizda “To'lovlar tarixi” bo'limi bor. U bo'limda CLICK tizimi orqali amalga oshirilgan to'lovlaringiz tarixi ko'rstailadi. Agar kartangizning barcha kirim va chiqimlarini ko'rmoqchi bo'lsangiz. \n                 1. Ilova  (Click Evolution) \n?? Sozlamalar \nBildirishnomalar\nKartalar bo'yicha tarix \n                 2. Bot  (havola- clickuz)\n?? Tarix\nTo'lovlarni ko'rsatish\n                 3. Sayt  (my.click.uz) \n ?? Tarix \nKarta bo'yicha tarix \n               4. USSD  (*880#)  \n  *880*00*15# - Oxirgi 15 to'lovlar holatini tekshirish \n(Pullik xizmat. Mobil operatoringizga bog'liq)\n   *880*00*3#- Oxirgi 3 to'lovlar holatini tekshirish- bepul\n       Shu bo'lim orqali qilgan to'lov

In [18]:
df = data[data['answer_text'].isin(classess)].reset_index(drop=True).groupby('answer_text').head(600)
df = df.dropna()

In [19]:
df['id'].unique()

array([     5,      6,     15, ..., 150790, 151215, 151378], dtype=int64)

In [20]:
df

,id,feedback_text,answer_text
0,5,Töğirlab bera olasizlami,Assalomu alaykum. Savolingizni aniqroq yozib y...
1,6,Pul oʻtkazayotgandim pul oʻtyabdi deb turibdi,"Assalomu alaykum. Hurmatli mijoz, to'lovingiz ..."
2,15,Firibgarlar kim,Assalomu alaykum. Savolingizni aniqroq yozib y...
3,18,Mani shotimdan pul yecildi digan xabar keldi l...,"Assalomu alaykum. Xozir siz bilan bog'lanib, y..."
4,23,Pul tushmadi ku,Assalomu alaykum. Savolingizni aniqroq yozib y...
...,...,...,...
15288,150688,Siz CLICK UZ Aksiyasida 3million Sóm Yutib Ol...,Assalomu alaykum! \nYuqoridagi ma'lumotlar biz...
15290,150757,bir firibgarga pul ko'chirdim ishni bajarib be...,Assalomu alaykum. Afsuski yoq.
15295,150790,Pubg mobile oyiniga 67 ming som miqdorda tolov...,Assalomu alaykum. To'lov raqamini yozib yubori...
15307,151215,https://xalqbank-uz.rasmiykredit.ru/card/21123...,Assalomu alaykum! \nYuqoridagi ma'lumotlar biz...


In [21]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-zа-яё #+_]')
# STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    # text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

df['feedback_text'] = df['feedback_text'].apply(clean_text)

 # df['answer_text'] = df['answer_text'].apply(lambda x: classess.index(x))

 # onehot = pd.get_dummies(df['id'])
df = df.drop('id', axis=1)
 # df = df.join(onehot)


In [22]:
df

,feedback_text,answer_text
0,tirlab bera olasizlami,Assalomu alaykum. Savolingizni aniqroq yozib y...
1,pul otkazayotgandim pul otyabdi deb turibdi,"Assalomu alaykum. Hurmatli mijoz, to'lovingiz ..."
2,firibgarlar kim,Assalomu alaykum. Savolingizni aniqroq yozib y...
3,mani shotimdan pul yecildi digan xabar keldi l...,"Assalomu alaykum. Xozir siz bilan bog'lanib, y..."
4,pul tushmadi ku,Assalomu alaykum. Savolingizni aniqroq yozib y...
...,...,...
15288,siz click uz aksiyasida 3million sm yutib old...,Assalomu alaykum! \nYuqoridagi ma'lumotlar biz...
15290,bir firibgarga pul kochirdim ishni bajarib ber...,Assalomu alaykum. Afsuski yoq.
15295,pubg mobile oyiniga 67 ming som miqdorda tolov...,Assalomu alaykum. To'lov raqamini yozib yubori...
15307,https xalqbankuzrasmiykreditru card 211237061,Assalomu alaykum! \nYuqoridagi ma'lumotlar biz...


In [23]:
len(df)

10536

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
vectorizer = TfidfVectorizer()

In [26]:
X = vectorizer.fit_transform(df['feedback_text'])
y = df['answer_text']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

## CatBoost

In [28]:
from catboost import CatBoostClassifier, Pool

In [29]:
X = vectorizer.fit_transform(df['feedback_text'])
y = df['answer_text']


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [31]:
test_data = catboost_pool = Pool(X_test,
                                 y_test)

In [32]:
tunned_model = CatBoostClassifier(
    random_seed=70,
    depth=5,
    loss_function='MultiClass',
    iterations=10000,
    learning_rate=0.05,
    l2_leaf_reg=3,
    bagging_temperature=1,
    random_strength=1,
    one_hot_max_size=2,
    leaf_estimation_method='Newton' 
)

tunned_model.fit(
    X_train, y_train,
    verbose=False,
    eval_set=(X_test, y_test),

)

In [77]:
tunned_model.score(X_test, y_test)

0.6927777777777778

In [78]:
best_model = CatBoostClassifier(
    random_seed=63,
    iterations=int(tunned_model.tree_count_ * 1.2)
)

best_model.fit(
    X, y,
    verbose=100
)

Learning rate set to 0.010619
0:	learn: 2.2907181	total: 335ms	remaining: 1h 7m 1s
100:	learn: 1.8822084	total: 40.7s	remaining: 1h 19m 59s
200:	learn: 1.7315878	total: 1m 26s	remaining: 1h 24m 45s
300:	learn: 1.6410662	total: 2m 17s	remaining: 1h 29m 3s
400:	learn: 1.5732865	total: 3m 12s	remaining: 1h 32m 55s
500:	learn: 1.5195992	total: 4m 2s	remaining: 1h 32m 54s
600:	learn: 1.4747859	total: 4m 56s	remaining: 1h 33m 41s
700:	learn: 1.4358809	total: 5m 54s	remaining: 1h 35m 17s
800:	learn: 1.4011466	total: 6m 47s	remaining: 1h 35m 9s
900:	learn: 1.3690530	total: 7m 39s	remaining: 1h 34m 29s
1000:	learn: 1.3409034	total: 8m 21s	remaining: 1h 32m 1s
1100:	learn: 1.3158184	total: 9m 5s	remaining: 1h 30m 6s
1200:	learn: 1.2929751	total: 9m 51s	remaining: 1h 28m 42s
1300:	learn: 1.2720974	total: 10m 40s	remaining: 1h 27m 50s
1400:	learn: 1.2532731	total: 11m 33s	remaining: 1h 27m 29s
1500:	learn: 1.2363737	total: 12m 29s	remaining: 1h 27m 24s
1600:	learn: 1.2209476	total: 13m 28s	remaini

In [79]:
best_model.score(X_test, y_test)

0.8172222222222222

In [80]:
test_pool = Pool(X_test)
predictions=best_model.predict_proba(test_pool)
print(f"Predictions: {predictions}")

Predictions: [[0.25773717 0.12248449 0.05969313 ... 0.08099299 0.07616907 0.10498618]
 [0.01996764 0.01440993 0.00772766 ... 0.0076476  0.88190683 0.02609673]
 [0.29162734 0.10964077 0.06036734 ... 0.0849942  0.0696594  0.09066354]
 ...
 [0.08201032 0.06946435 0.02893892 ... 0.01834221 0.03133357 0.03833094]
 [0.31414284 0.08882619 0.13778995 ... 0.02850311 0.01701338 0.03181671]
 [0.35320891 0.1116064  0.05177317 ... 0.0751953  0.05902144 0.08134473]]


In [81]:
# make the prediction using the resulting model
preds_class = best_model.predict(test_data)
preds_proba = best_model.predict_proba(test_data)

In [82]:
best_model.save_model('model_name')